# [Hands-On] N-gram 생성 및 BOW 벡터화 기초

- Author: Sangkeun Jung (hugmanskj@gmail.com)

> 교육 목적

**Copyright**: All rights reserved

---

## 개요

N-gram의 개념을 이해하고, BOW(Bag-of-Words) 벡터화 기법을 실습합니다.


**학습 목표**:
1. Unigram, Bigram, Trigram 생성 방법 이해
2. N-gram을 이용한 BOW 벡터 생성
3. 문장을 고정 차원의 벡터로 표현하는 방법 습득
4. N-gram이 문맥 정보를 어떻게 보존하는지 이해

---

## 1. 라이브러리 로드

In [ ]:
from collections import Counter
from typing import List, Dict, Tuple
import numpy as np

print("라이브러리 로드 완료!")

라이브러리 로드 완료!


---

## 2. N-gram 생성 함수 구현

### 2.1 N-gram 생성 함수

토큰 리스트로부터 n-gram을 생성하는 함수를 구현합니다.

In [ ]:
def generate_ngrams(tokens, n):
    """
    토큰 리스트로부터 n-gram을 생성합니다.

    Args:
        tokens: 토큰 리스트 (예: ['나는', '밥을', '먹었다'])
        n: n-gram의 크기 (1=unigram, 2=bigram, 3=trigram)

    Returns:
        n-gram 리스트 (예: n=2일 때 ['나는 밥을', '밥을 먹었다'])

    예시:
        tokens = ['나는', '밥을', '먹었다']
        generate_ngrams(tokens, 1) → ['나는', '밥을', '먹었다']
        generate_ngrams(tokens, 2) → ['나는 밥을', '밥을 먹었다']
        generate_ngrams(tokens, 3) → ['나는 밥을 먹었다']
    """
    ngrams = []


    for i in range(len(tokens) - n + 1):
        ngram = ' '.join(tokens[i:i+n])
        ngrams.append(ngram)

    return ngrams

In [ ]:
# 함수 테스트
test_tokens = ['나는', '밥을', '먹었다']

print("=" * 70)
print("N-gram 생성 테스트")
print("=" * 70)
print("\n입력 토큰:", test_tokens)
print("\nUnigram (n=1):", generate_ngrams(test_tokens, 1))
print("Bigram (n=2):", generate_ngrams(test_tokens, 2))
print("Trigram (n=3):", generate_ngrams(test_tokens, 3))

N-gram 생성 테스트

입력 토큰: ['나는', '밥을', '먹었다']

Unigram (n=1): ['나는', '밥을', '먹었다']
Bigram (n=2): ['나는 밥을', '밥을 먹었다']
Trigram (n=3): ['나는 밥을 먹었다']


### 2.2 Vocabulary 생성 함수

여러 문장으로부터 전체 vocabulary를 생성합니다.

In [ ]:
def create_vocabulary(sentences, ngram_range=(1, 2)):
    """
    여러 문장으로부터 vocabulary를 생성합니다.

    Args:
        sentences: 토큰화된 문장들의 리스트
        ngram_range: n-gram 범위 (min_n, max_n)
                     예: (1, 2)는 unigram과 bigram 모두 포함

    Returns:
        정렬된 vocabulary 리스트

    예시:
        sentences = [['강한', '비가', '내렸다'], ['비가', '많이', '내렸다']]
        create_vocabulary(sentences, (1, 2))
        → ['강한', '내렸다', '많이', '비가', '강한 비가', '비가 내렸다', '비가 많이']
    """
    all_ngrams = set()

    min_n, max_n = ngram_range

    for tokens in sentences:
        for n in range(min_n, max_n + 1):
            ngrams = generate_ngrams(tokens, n)
            all_ngrams.update(ngrams)

    # 정렬하여 일관된 순서 유지
    vocabulary = sorted(all_ngrams)
    return vocabulary

In [ ]:
# 함수 테스트
test_sentences = [
    ['강한', '비가', '내렸다'],
    ['비가', '많이', '내렸다']
]

vocab = create_vocabulary(test_sentences, ngram_range=(1, 2))

print("=" * 70)
print("Vocabulary 생성 테스트")
print("=" * 70)
print("\n입력 문장:")
for i, sent in enumerate(test_sentences, 1):
    print("  %d. %s" % (i, ' '.join(sent)))

print("\n생성된 Vocabulary (%d개):" % len(vocab))
for i, feature in enumerate(vocab, 1):
    print("  %2d. %s" % (i, feature))

Vocabulary 생성 테스트

입력 문장:
  1. 강한 비가 내렸다
  2. 비가 많이 내렸다

생성된 Vocabulary (8개):
   1. 강한
   2. 강한 비가
   3. 내렸다
   4. 많이
   5. 많이 내렸다
   6. 비가
   7. 비가 내렸다
   8. 비가 많이


### 2.3 BOW 벡터 변환 함수

토큰 리스트를 BOW 벡터로 변환합니다.

In [ ]:
def text_to_bow_vector(tokens, vocabulary, ngram_range=(1, 2)):
    """
    토큰 리스트를 BOW 벡터로 변환합니다.

    Args:
        tokens: 토큰 리스트
        vocabulary: 전체 어휘 리스트
        ngram_range: n-gram 범위

    Returns:
        BOW 벡터 (vocabulary 크기와 동일한 차원)

    예시:
        tokens = ['강한', '비가', '내렸다']
        vocabulary = ['강한', '비가', '내렸다', '많이', '강한 비가', '비가 내렸다']
        vector = text_to_bow_vector(tokens, vocabulary, (1, 2))
        → [1, 1, 1, 0, 1, 1]  (각 feature의 출현 횟수)
    """
    # vocabulary를 인덱스로 매핑
    vocab_to_idx = {word: idx for idx, word in enumerate(vocabulary)}

    # 0으로 초기화된 벡터 생성
    vector = np.zeros(len(vocabulary), dtype=int)

    min_n, max_n = ngram_range

    for n in range(min_n, max_n + 1):
        ngrams = generate_ngrams(tokens, n)
        for ngram in ngrams:
            if ngram in vocab_to_idx:
                idx = vocab_to_idx[ngram]
                vector[idx] += 1

    return vector

In [ ]:
# 함수 테스트
test_tokens = ['강한', '비가', '내렸다']
test_vocab = ['강한', '비가', '내렸다', '많이', '강한 비가', '비가 내렸다', '비가 많이']

vector = text_to_bow_vector(test_tokens, test_vocab, ngram_range=(1, 2))

print("=" * 70)
print("BOW 벡터 변환 테스트")
print("=" * 70)
print("\n입력 토큰:", test_tokens)
print("\nVocabulary:")
for i, feat in enumerate(test_vocab):
    print("  %d. %s" % (i, feat))

print("\n생성된 벡터:", list(vector))
print("\nFeature 출현 횟수:")
for i, count in enumerate(vector):
    if count > 0:
        print("  %s: %d회" % (test_vocab[i], count))

BOW 벡터 변환 테스트

입력 토큰: ['강한', '비가', '내렸다']

Vocabulary:
  0. 강한
  1. 비가
  2. 내렸다
  3. 많이
  4. 강한 비가
  5. 비가 내렸다
  6. 비가 많이

생성된 벡터: [np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0)]

Feature 출현 횟수:
  강한: 1회
  비가: 1회
  내렸다: 1회
  강한 비가: 1회
  비가 내렸다: 1회


---

## 3. 실습 예제 1: 슬라이드 예제 재현 (Unigram + Bigram)

**문장들**:
- 문장 1: "강한 비가 내렸다"
- 문장 2: "비가 많이 내렸다"
- 문장 3: "어제 강한 바람과 비가 내렸다"

In [ ]:
print("=" * 70)
print("실습 예제 1: 슬라이드 예제 재현 (Unigram + Bigram)")
print("=" * 70)

# 토큰화된 문장들 (띄어쓰기 기준)
sentences = [
    ['강한', '비가', '내렸다'],
    ['비가', '많이', '내렸다'],
    ['어제', '강한', '바람과', '비가', '내렸다']
]

print("\n[입력 문장들]")
for i, sent in enumerate(sentences, 1):
    print("문장 %d: %s" % (i, ' '.join(sent)))

실습 예제 1: 슬라이드 예제 재현 (Unigram + Bigram)

[입력 문장들]
문장 1: 강한 비가 내렸다
문장 2: 비가 많이 내렸다
문장 3: 어제 강한 바람과 비가 내렸다


In [ ]:
# Step 1: Unigram 생성
print("\n[Step 1] Unigram 생성")
for i, tokens in enumerate(sentences, 1):
    unigrams = generate_ngrams(tokens, n=1)
    print("문장 %d: %s" % (i, str(unigrams)))


[Step 1] Unigram 생성
문장 1: ['강한', '비가', '내렸다']
문장 2: ['비가', '많이', '내렸다']
문장 3: ['어제', '강한', '바람과', '비가', '내렸다']


In [ ]:
# Step 2: Bigram 생성
print("\n[Step 2] Bigram 생성")
for i, tokens in enumerate(sentences, 1):
    bigrams = generate_ngrams(tokens, n=2)
    print("문장 %d: %s" % (i, str(bigrams)))


[Step 2] Bigram 생성
문장 1: ['강한 비가', '비가 내렸다']
문장 2: ['비가 많이', '많이 내렸다']
문장 3: ['어제 강한', '강한 바람과', '바람과 비가', '비가 내렸다']


In [ ]:
# Step 3: Vocabulary 생성 (Unigram + Bigram)
print("\n[Step 3] 전체 Vocabulary 생성 (Unigram + Bigram)")
vocabulary = create_vocabulary(sentences, ngram_range=(1, 2))
print("총 %d개의 feature" % len(vocabulary))
print("\nVocabulary:")
for idx, feature in enumerate(vocabulary):
    print("  %2d. %s" % (idx, feature))


[Step 3] 전체 Vocabulary 생성 (Unigram + Bigram)
총 13개의 feature

Vocabulary:
   0. 강한
   1. 강한 바람과
   2. 강한 비가
   3. 내렸다
   4. 많이
   5. 많이 내렸다
   6. 바람과
   7. 바람과 비가
   8. 비가
   9. 비가 내렸다
  10. 비가 많이
  11. 어제
  12. 어제 강한


In [ ]:
# Step 4: 각 문장을 벡터로 변환
print("\n[Step 4] 각 문장을 13차원 벡터로 표현")
print("\n(슬라이드 14페이지 참조)")

for i, tokens in enumerate(sentences, 1):
    vector = text_to_bow_vector(tokens, vocabulary, ngram_range=(1, 2))
    print("\n문장 %d 벡터: %s" % (i, list(vector)))
    print("  원문: %s" % ' '.join(tokens))

    # 어떤 feature가 포함되었는지 출력
    print("  포함된 features:")
    for idx, count in enumerate(vector):
        if count > 0:
            print("    - %s: %d회" % (vocabulary[idx], count))


[Step 4] 각 문장을 13차원 벡터로 표현

(슬라이드 14페이지 참조)

문장 1 벡터: [np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0)]
  원문: 강한 비가 내렸다
  포함된 features:
    - 강한: 1회
    - 강한 비가: 1회
    - 내렸다: 1회
    - 비가: 1회
    - 비가 내렸다: 1회

문장 2 벡터: [np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0)]
  원문: 비가 많이 내렸다
  포함된 features:
    - 내렸다: 1회
    - 많이: 1회
    - 많이 내렸다: 1회
    - 비가: 1회
    - 비가 많이: 1회

문장 3 벡터: [np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1)]
  원문: 어제 강한 바람과 비가 내렸다
  포함된 features:
    - 강한: 1회
    - 강한 바람과: 1회
    - 내렸다: 1회
    - 바람과: 1회
    - 바람과 비가: 1회
    - 비가: 1회
    - 비가 내렸다: 1회
    - 어제: 1회
    - 어제 강한: 1회


---

## 4. 실습 예제 2: 영어 문장으로 연습

영어 예제로 추가 연습을 진행합니다.

In [ ]:
print("=" * 70)
print("실습 예제 2: 영어 문장으로 연습")
print("=" * 70)

sentences_en = [
    ['i', 'love', 'machine', 'learning'],
    ['machine', 'learning', 'is', 'fun'],
    ['i', 'study', 'machine', 'learning']
]

print("\n[입력 문장들]")
for i, sent in enumerate(sentences_en, 1):
    print("문장 %d: %s" % (i, ' '.join(sent)))

실습 예제 2: 영어 문장으로 연습

[입력 문장들]
문장 1: i love machine learning
문장 2: machine learning is fun
문장 3: i study machine learning


In [ ]:
# Unigram만 사용
print("\n--- Unigram만 사용한 경우 ---")
vocab_unigram = create_vocabulary(sentences_en, ngram_range=(1, 1))
print("Vocabulary (크기 %d): %s" % (len(vocab_unigram), str(vocab_unigram)))

for i, tokens in enumerate(sentences_en, 1):
    vector = text_to_bow_vector(tokens, vocab_unigram, ngram_range=(1, 1))
    print("문장 %d 벡터: %s" % (i, list(vector)))


--- Unigram만 사용한 경우 ---
Vocabulary (크기 7): ['fun', 'i', 'is', 'learning', 'love', 'machine', 'study']
문장 1 벡터: [np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0)]
문장 2 벡터: [np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0)]
문장 3 벡터: [np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1)]


In [ ]:
# Unigram + Bigram 사용
print("\n--- Unigram + Bigram 사용한 경우 ---")
vocab_bigram = create_vocabulary(sentences_en, ngram_range=(1, 2))
print("Vocabulary (크기 %d): %s" % (len(vocab_bigram), str(vocab_bigram)))

for i, tokens in enumerate(sentences_en, 1):
    vector = text_to_bow_vector(tokens, vocab_bigram, ngram_range=(1, 2))
    print("문장 %d 벡터: %s" % (i, list(vector)))

print("\n✓ Bigram을 추가하면 vocabulary 크기가 증가합니다.")
print("  Unigram만: %d개" % len(vocab_unigram))
print("  Unigram+Bigram: %d개" % len(vocab_bigram))
print("✓ 하지만 문맥 정보가 더 잘 반영됩니다!")


--- Unigram + Bigram 사용한 경우 ---
Vocabulary (크기 14): ['fun', 'i', 'i love', 'i study', 'is', 'is fun', 'learning', 'learning is', 'love', 'love machine', 'machine', 'machine learning', 'study', 'study machine']
문장 1 벡터: [np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0)]
문장 2 벡터: [np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0)]
문장 3 벡터: [np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1)]

✓ Bigram을 추가하면 vocabulary 크기가 증가합니다.
  Unigram만: 7개
  Unigram+Bigram: 14개
✓ 하지만 문맥 정보가 더 잘 반영됩니다!


---

## 5. 연습 문제 1: 다양한 N-gram 비교

같은 문장들을 Unigram, Bigram, Trigram, Unigram+Bigram으로
각각 벡터화하고 결과를 비교해보세요.

In [ ]:
print("=" * 70)
print("연습 문제 1: 다양한 N-gram 설정 비교")
print("=" * 70)

sentences_kr = [
    ['자연어', '처리는', '재미있다'],
    ['기계', '학습은', '유용하다'],
    ['자연어', '처리와', '기계', '학습']
]

print("\n[입력 문장들]")
for i, sent in enumerate(sentences_kr, 1):
    print("문장 %d: %s" % (i, ' '.join(sent)))

# TODO: 다음 설정들로 각각 vocabulary를 만들고 벡터화 결과를 비교하세요
# 1. Unigram only: (1, 1)
# 2. Bigram only: (2, 2)
# 3. Trigram only: (3, 3)
# 4. Unigram + Bigram: (1, 2)
# 5. Unigram + Bigram + Trigram: (1, 3)

연습 문제 1: 다양한 N-gram 설정 비교

[입력 문장들]
문장 1: 자연어 처리는 재미있다
문장 2: 기계 학습은 유용하다
문장 3: 자연어 처리와 기계 학습


In [ ]:
configurations = [
    ((1, 1), "Unigram only"),
    ((2, 2), "Bigram only"),
    ((3, 3), "Trigram only"),
    ((1, 2), "Unigram + Bigram"),
    ((1, 3), "Unigram + Bigram + Trigram")
]

for ngram_range, name in configurations:
    print("\n--- %s ---" % name)
    vocab = create_vocabulary(sentences_kr, ngram_range=ngram_range)
    print("Vocabulary 크기: %d" % len(vocab))

    if len(vocab) > 10:
        print("Features: %s ..." % str(vocab[:10]))
    else:
        print("Features: %s" % str(vocab))

    # 첫 번째 문장만 벡터화
    vector = text_to_bow_vector(sentences_kr[0], vocab, ngram_range=ngram_range)
    print("문장 1 벡터 (non-zero만): ", end="")
    non_zero = [(vocab[i], int(vector[i])) for i in range(len(vector)) if vector[i] > 0]
    print(non_zero)


--- Unigram only ---
Vocabulary 크기: 8
Features: ['기계', '유용하다', '자연어', '재미있다', '처리는', '처리와', '학습', '학습은']
문장 1 벡터 (non-zero만): [('자연어', 1), ('재미있다', 1), ('처리는', 1)]

--- Bigram only ---
Vocabulary 크기: 7
Features: ['기계 학습', '기계 학습은', '자연어 처리는', '자연어 처리와', '처리는 재미있다', '처리와 기계', '학습은 유용하다']
문장 1 벡터 (non-zero만): [('자연어 처리는', 1), ('처리는 재미있다', 1)]

--- Trigram only ---
Vocabulary 크기: 4
Features: ['기계 학습은 유용하다', '자연어 처리는 재미있다', '자연어 처리와 기계', '처리와 기계 학습']
문장 1 벡터 (non-zero만): [('자연어 처리는 재미있다', 1)]

--- Unigram + Bigram ---
Vocabulary 크기: 15
Features: ['기계', '기계 학습', '기계 학습은', '유용하다', '자연어', '자연어 처리는', '자연어 처리와', '재미있다', '처리는', '처리는 재미있다'] ...
문장 1 벡터 (non-zero만): [('자연어', 1), ('자연어 처리는', 1), ('재미있다', 1), ('처리는', 1), ('처리는 재미있다', 1)]

--- Unigram + Bigram + Trigram ---
Vocabulary 크기: 19
Features: ['기계', '기계 학습', '기계 학습은', '기계 학습은 유용하다', '유용하다', '자연어', '자연어 처리는', '자연어 처리는 재미있다', '자연어 처리와', '자연어 처리와 기계'] ...
문장 1 벡터 (non-zero만): [('자연어', 1), ('자연어 처리는', 1), ('자연어 처리는 재미있다', 1), ('재미있다', 1), ('처리는

---

## 6. 연습 문제 2: 문장 유사도 계산

두 문장의 BOW 벡터를 만들고, 코사인 유사도를 계산하여
문장 간의 유사도를 측정해보세요.

In [ ]:
def cosine_similarity(vec1, vec2):
    """코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

In [ ]:
print("=" * 70)
print("연습 문제 2: 문장 유사도 계산")
print("=" * 70)

# 세 개의 문장
sentences_sim = [
    ['날씨가', '좋다'],
    ['날씨가', '매우', '좋다'],
    ['비가', '온다']
]

print("\n[입력 문장들]")
for i, sent in enumerate(sentences_sim, 1):
    print("문장 %d: %s" % (i, ' '.join(sent)))

연습 문제 2: 문장 유사도 계산

[입력 문장들]
문장 1: 날씨가 좋다
문장 2: 날씨가 매우 좋다
문장 3: 비가 온다


In [ ]:
# Vocabulary 생성 및 벡터화
vocab = create_vocabulary(sentences_sim, ngram_range=(1, 2))
vectors = [text_to_bow_vector(sent, vocab, (1, 2)) for sent in sentences_sim]

print("\nVocabulary: %s" % str(vocab))


Vocabulary: ['날씨가', '날씨가 매우', '날씨가 좋다', '매우', '매우 좋다', '비가', '비가 온다', '온다', '좋다']


In [ ]:
# 모든 문장 쌍의 유사도 계산
print("\n[문장 간 유사도]")
for i in range(len(sentences_sim)):
    for j in range(i+1, len(sentences_sim)):
        sim = cosine_similarity(vectors[i], vectors[j])
        print("문장 %d vs 문장 %d: %.4f" % (i+1, j+1, sim))
        print("  '%s' vs '%s'" % (' '.join(sentences_sim[i]), ' '.join(sentences_sim[j])))

print("\n✓ '날씨가 좋다'와 '날씨가 매우 좋다'는 높은 유사도를 가집니다.")
print("✓ '비가 온다'는 다른 문장들과 낮은 유사도를 가집니다.")


[문장 간 유사도]
문장 1 vs 문장 2: 0.5164
  '날씨가 좋다' vs '날씨가 매우 좋다'
문장 1 vs 문장 3: 0.0000
  '날씨가 좋다' vs '비가 온다'
문장 2 vs 문장 3: 0.0000
  '날씨가 매우 좋다' vs '비가 온다'

✓ '날씨가 좋다'와 '날씨가 매우 좋다'는 높은 유사도를 가집니다.
✓ '비가 온다'는 다른 문장들과 낮은 유사도를 가집니다.


---

## 7. 실습 요약

### 배운 내용
1. **N-gram 생성**: Unigram, Bigram, Trigram
2. **Vocabulary 구축**: 여러 문장으로부터 통합 어휘 생성
3. **BOW 벡터화**: 문장을 고정 차원 숫자 벡터로 변환
4. **문장 유사도**: 코사인 유사도를 이용한 문장 비교

### 주요 발견
- N-gram을 추가하면 vocabulary 크기가 증가하지만 문맥 정보가 보존됨
- Bigram은 인접한 두 단어의 관계를 포착
- BOW 벡터를 이용하면 문장 간 유사도를 계산할 수 있음

### 다음 단계
- **실습 2**: 텍스트 전처리 파이프라인 구축
- **실습 3**: 실제 의료 판독문 데이터 분석